In [18]:
import pandas as pd
dfans = pd.read_csv('output_val/GraderExperiment_valGTpanelindex.csv',index_col=0)
dfHim = pd.read_csv('output_val/GraderExperiment_Val_HK.csv',sep='\t',index_col='ScanID')
dfimg = pd.read_csv('output_val/dfimg_val.csv',index_col=0)
dfimg_bug = pd.read_csv('output_val/dfimg_val_mergebug.csv',index_col=0)

marks = (dfHim['Prediction']==dfans['panel_index'])
print('{} out of {} correct'.format(marks.sum(),len(marks)))
align_w_dt = dfimg.loc[dfans[~marks]['ImgId']]
align_w_gt = dfimg.loc[dfans[marks]['ImgId']]
dfans[~marks].index


36 out of 50 correct


Int64Index([1, 11, 12, 13, 15, 16, 17, 24, 26, 28, 33, 34, 36, 38], dtype='int64')

In [21]:
from plain_train_net import CreatePlotsRPD
pltcrr = CreatePlotsRPD(align_w_gt)
plticrr = CreatePlotsRPD(align_w_dt)
rat = (align_w_dt.gt_instances<align_w_dt.dt_instances)
print('Ratio of dt>gt in align_w_dt: {}:{}'.format(rat.sum(),(len(rat)-rat.sum())))
rat = (align_w_gt.gt_instances<align_w_gt.dt_instances)
print('Ratio of dt>gt in align_w_gt: {}:{}'.format(rat.sum(),(len(rat)-rat.sum())))

(align_w_gt.gt_instances==align_w_gt.dt_instances).sum()


Ratio of dt>gt in align_w_dt: 8:6
Ratio of dt>gt in align_w_gt: 15:21


0

In [22]:
from detectron2.data import DatasetCatalog,MetadataCatalog
from plain_train_net import grab_dataset, OutputVis
import os

dataset_name = "val"
for name in [dataset_name]:
    try:
        DatasetCatalog.register(name, grab_dataset(name))
    except:
        print('Already registered.')
        #do nothing
    MetadataCatalog.get(name).thing_classes = ["rpd"]

pred_file = "output_"+ dataset_name + "/coco_instances_results.json"
out_file = os.path.join("output_"+ dataset_name,'GraderExperiment_'+dataset_name+'Him_align_w_dt_fixed.pdf')

vis = OutputVis(dataset_name,prob_thresh = 0.5,pred_mode='file',pred_file=pred_file,has_annotations=True)
ImgIds = align_w_dt.index.values
vis.output_to_pdf(ImgIds,out_file,dfimg=align_w_dt)

100%|██████████| 14/14 [00:22<00:00,  1.63s/it]


In [15]:
dfb = dfimg_bug.dt_instances.loc[dfimg.index]
df = dfimg.dt_instances
idx = ~(dfb==df)
dfb[idx].to_frame().join(df[idx],lsuffix='_b')


,dt_instances_b,dt_instances
101273_OS_oct-009.png,5.0,4.0
101243_OD_oct-000.png,4.0,3.0
101308_OS_oct-005.png,8.0,6.0
101135_OD_oct-007.png,9.0,5.0
101135_OS_oct-022.png,6.0,4.0
...,...,...
101243_OS_oct-013.png,5.0,3.0
101135_OD_oct-038.png,15.0,12.0
101273_OS_oct-043.png,13.0,10.0
101135_OD_oct-044.png,11.0,9.0
